In [59]:
import torch
import pandas as pd

from algorithms import LogisticRegressionRBF
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x, y = make_classification(n_samples=1000, n_features=3, n_informative=3, n_redundant=0, random_state=1)
x = torch.FloatTensor(x)
y = torch.FloatTensor(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [60]:
model = LogisticRegressionRBF(x_train, rbf='gaussian')
model = model.fit(x_train, y_train, 3000)

Epoch:     2 | CrossEntropyLoss:  0.67645
Epoch:     4 | CrossEntropyLoss:  0.64699
Epoch:     8 | CrossEntropyLoss:  0.59243
Epoch:    16 | CrossEntropyLoss:  0.50061
Epoch:    29 | CrossEntropyLoss:  0.39448
Epoch:    53 | CrossEntropyLoss:  0.28882
Epoch:    96 | CrossEntropyLoss:  0.21307
Epoch:   170 | CrossEntropyLoss:  0.16429
Epoch:   303 | CrossEntropyLoss:  0.13081
Epoch:   538 | CrossEntropyLoss:  0.10816
Epoch:   955 | CrossEntropyLoss:  0.09265
Epoch:  1692 | CrossEntropyLoss:  0.08161
Epoch:  3000 | CrossEntropyLoss:  0.07120


## Testing

In [61]:
metrics = model.metrics_tab(x_test, y_test)
tab = pd.DataFrame({'0': metrics['0.0'], '1': metrics['1.0']}).T.round(3)
print(f'Accuracy: {metrics["accuracy"]: 0.2%}')
tab

Accuracy:  94.33%


,precision,recall,f1-score,support
0,0.961,0.931,0.946,159.0
1,0.925,0.957,0.941,141.0


In [62]:
from metrics import roc_curve_plot, auc_roc

y_prob = model(x_test).detach().cpu().numpy().flatten()
print(f'AUC-ROC: {auc_roc(y_test, y_prob)}')
roc_curve_plot(y_test, y_prob, fill=True)

AUC-ROC: 0.9841652170034346


## Visualization of initial data

In [63]:
from sklearn.manifold import TSNE
import plotly.express as px

colors = list(map(lambda e: str(int(e)), y))

if x.shape[1] <= 2:
    fig = px.scatter(x=x[:, 0], y=x[:, 1], title='<b>Initial Distribution</b>', color=colors)

else:
    tsne_x = TSNE().fit_transform(x)
    fig = px.scatter(x=tsne_x[:, 0], y=tsne_x[:, 1], title='<b>TSNE of Distribution</b>', color=colors)

fig.show()

In [64]:
phi = model.make_phi_matrix(x)

if phi.shape[1] <= 2:
    fig = px.scatter(x=phi[:, 0], y=phi[:, 1], title='<b>Distribution after making Phi-Matrix</b>', color=colors)

else:
    tsne_phi = TSNE().fit_transform(phi)
    fig = px.scatter(x=tsne_phi[:, 0], y=tsne_phi[:, 1], title='<b>TSNE of Distribution after making Phi-Matrix</b>', color=colors)

fig.show()